# YouTube Link - https://youtu.be/P7gRmcb7y8Q

# **Import Dataset**

In [8]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import random
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.mlab as mlab
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
random.seed(42)

#Mount your Google drive 
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import fileinput

filePath ="/content/gdrive/My Drive/3253 Data Science/Framingham Dataset/framingham.csv"

with fileinput.FileInput(filePath, inplace=True, backup='.bak') as file:
    for line in file:
        print(line.replace(",", ";"), end='')

In [0]:
# Create dataframe
import pandas as pd
data = pd.read_csv(filePath, sep=";", index_col=0)

In [11]:
data.shape

(4240, 15)

In [12]:
#see if there are any missing values in the dataset
data.isnull().sum()

#drop the missing data
data.dropna(axis=0,inplace=True)

#check the drop worked 
data.isnull().sum()


age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64

In [13]:
#look at the dataset again
data.describe()

,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
count,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000
mean,49.551941,1.980317,0.489065,9.025424,0.030344,0.005741,0.311646,0.027064,236.847731,132.370558,82.917031,25.782802,75.730727,81.852925,0.152269
std,8.562029,1.022656,0.499949,11.921590,0.171557,0.075561,0.463229,0.162292,44.097681,22.086866,11.974258,4.065601,11.981525,23.904164,0.359331
min,32.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000,83.500000,48.000000,15.540000,44.000000,40.000000,0.000000
25%,42.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,206.000000,117.000000,75.000000,23.080000,68.000000,71.000000,0.000000
50%,49.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,234.000000,128.000000,82.000000,25.380000,75.000000,78.000000,0.000000
75%,56.000000,3.000000,1.000000,20.000000,0.000000,0.000000,1.000000,0.000000,263.000000,143.875000,90.000000,28.037500,82.000000,87.000000,0.000000
max,70.000000,4.000000,1.000000,70.000000,1.000000,1.000000,1.000000,1.000000,600.000000,295.000000,142.500000,56.800000,143.000000,394.000000,1.000000


#**Logistic Regression**

In [14]:
from statsmodels.tools import add_constant as add_constant
data_constant = add_constant(data)
data_constant.head()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,const,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
male,,,,,,,,,,,,,,,,
1,1.0,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
0,1.0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
1,1.0,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
0,1.0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
0,1.0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [15]:
from statsmodels.tools import add_constant as add_constant
heart_df_constant = add_constant(data)
heart_df_constant.head()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,const,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
male,,,,,,,,,,,,,,,,
1,1.0,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
0,1.0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
1,1.0,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
0,1.0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
0,1.0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [16]:
st.chisqprob = lambda chisq, df: st.chi2.sf(chisq, df)
cols=heart_df_constant.columns[:-1]
model=sm.Logit(data.TenYearCHD,heart_df_constant[cols])
result=model.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.380070
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             TenYearCHD   No. Observations:                 3658
Model:                          Logit   Df Residuals:                     3643
Method:                           MLE   Df Model:                           14
Date:                Sun, 18 Aug 2019   Pseudo R-squ.:                  0.1091
Time:                        22:22:53   Log-Likelihood:                -1390.3
converged:                       True   LL-Null:                       -1560.6
Covariance Type:            nonrobust   LLR p-value:                 3.860e-64
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -7.8757      0.702    -11.214      0.000      -9.252      -6.499
age                 0.0656      0.007      9.790      0.000       0.052       0.079
education          -0.0458      0.050     -0.922      0.356      -0.143       0.052
currentSmoker       0.0635      0.155      0.409      0.683      -0.241       0.368
cigsPerDay          0.0261      0.006      4.357      0.000       0.014       0.038
BPMeds              0.1123      0.233      0.482      0.630      -0.345       0.569
prevalentStroke     0.7085      0.490      1.447      0.148      -0.251       1.668
prevalentHyp        0.2515      0.138      1.820      0.069      -0.019       0.522
diabetes            0.0822      0.315      0.261      0.794      -0.536       0.700
totChol             0.0016      0.001      1.413      0.158      -0.001       0.004
sysBP               0.0125      0.004      3.336      0.001       0.005       0.020
diaBP           -2.039e-05      0.006     -0.003      0.997      -0.013       0.012
BMI                 0.0084      0.013      0.671      0.502      -0.016       0.033
heartRate          -0.0059      0.004     -1.401      0.161      -0.014       0.002
glucose             0.0072      0.002      3.249      0.001       0.003       0.012
===================================================================================
"""

In [17]:
result.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             TenYearCHD   No. Observations:                 3658
Model:                          Logit   Df Residuals:                     3643
Method:                           MLE   Df Model:                           14
Date:                Sun, 18 Aug 2019   Pseudo R-squ.:                  0.1091
Time:                        22:22:53   Log-Likelihood:                -1390.3
converged:                       True   LL-Null:                       -1560.6
Covariance Type:            nonrobust   LLR p-value:                 3.860e-64
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -7.8757      0.702    -11.214      0.000      -9.252      -6.499
age                 0.0656      0.007      9.790      0.000       0.052       0.079
education          -0.0458      0.050     -0.922      0.356      -0.143       0.052
currentSmoker       0.0635      0.155      0.409      0.683      -0.241       0.368
cigsPerDay          0.0261      0.006      4.357      0.000       0.014       0.038
BPMeds              0.1123      0.233      0.482      0.630      -0.345       0.569
prevalentStroke     0.7085      0.490      1.447      0.148      -0.251       1.668
prevalentHyp        0.2515      0.138      1.820      0.069      -0.019       0.522
diabetes            0.0822      0.315      0.261      0.794      -0.536       0.700
totChol             0.0016      0.001      1.413      0.158      -0.001       0.004
sysBP               0.0125      0.004      3.336      0.001       0.005       0.020
diaBP           -2.039e-05      0.006     -0.003      0.997      -0.013       0.012
BMI                 0.0084      0.013      0.671      0.502      -0.016       0.033
heartRate          -0.0059      0.004     -1.401      0.161      -0.014       0.002
glucose             0.0072      0.002      3.249      0.001       0.003       0.012
===================================================================================
"""

# Creating test/train model 

In [0]:
import sklearn
from sklearn.model_selection import train_test_split
new_features=data
x=new_features.iloc[:,:-1]
y=new_features.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.20,random_state=5)

In [19]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression()
logreg.fit(x_train,y_train)
y_pred=logreg.predict(x_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [20]:
#test model accuracy
sklearn.metrics.accuracy_score(y_test,y_pred)


0.8661202185792349

In [21]:
sklearn.metrics.roc_auc_score(y_test,y_pred)

0.5368691817736404